In [ ]:
import nbaScraper as ns
import actNetScraper as ans
from datetime import datetime, timedelta

leagues = ['nba', 'nhl']   # None or list ['nba', 'nhl', 'nfl', 'mlb', 'wnba'] # NONE looks for all sports
specified =  []  ####specific prop, *****only works with a single league in leagues

# day adjustment from today (date of running script), negative = dates into the past
dayJump = 0 
# date can be a list of dates if multiple need scraping 'YYYY-MM-DD'
# default is to only pull today or today + dayJump
dates = [(datetime.today() + timedelta(days=dayJump)).strftime('%Y-%m-%d')]
#dates = ['2024-01-05']#, '2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11']

database_export = True  # add all scrapes to database
store_locally = True    # add all scrapes to class variables
season_int = 2025 # int will be the final year of the schedule season
season_str = '2024-25'

# nba website and basketball referenece scrapers
scraper = ns.scraper(
    browser_path = '..\\..\\browser\\geckodriver.exe',
    database_export = database_export, 
    store_locally = store_locally,
    pymysql_conn_str =  None
)
# assigns the date of the last time code executed as today
today = scraper.meta_data['today_dt']

# looks up the actual date for the last regular season game date. this will be used to grab the data for the completed games on the date
scraper.get_last_game_date(season = season_str)
run_date = scraper.last_game_date
dateRange = [
    run_date, run_date
]

prop_scraper = ans.actNetScraper(
    browser_path = '..\\..\\browser\\geckodriver.exe',
    dates = dates,
    leagues = leagues,
    database_export = database_export, 
    store_locally = store_locally,
    config_path = '..\\..\\..\\..\\Notes-General\\config.txt',
    second_run = False
)

# turn to False if issues loading new players
#prop_scraper.update_players = False

# update the league list to only ones with games today
#leagues = prop_scraper.check_for_league_games(date_check = None, update_class_leagues_var = True)
print('scraping for', prop_scraper.leagues, 'on', prop_scraper.dates)

Last day with games played: 2025-03-12
scraping for ['nba', 'nhl'] on ['2025-03-13'] Last completed game date: 2025-03-12


In [2]:
print(today, 'run date...\n')
prop_scraper.scrape(
    sleep_secs = 3, 
    specific_props = specified,
    leagues_override = prop_scraper.leagues,
    an_state_code = 'BC'
)

prop_scraper.processScrapes(
    remove_dups = True,
    specific_props = specified
)

if prop_scraper.scrape_error_flag:
    print(prop_scraper.scrape_errors)
    #prop_scraper.tryMissingProps()


2025-03-12 run date...

scraping nba ...
original rows:  (1078, 20)
after dups removed:  (1078, 20)
[]
nba odds data loaded...
prop          ast  blk   pa   pr  pra  pts   ra  reb  sb  stl  threes
propId count  107  104  109  110  110  109  106  108  13  104      98
scraping nhl ...
original rows:  (698, 20)
after dups removed:  (698, 20)
[]
nhl odds data loaded...
prop          ast  ats  gs  gs1st  gs2plus  gs3plus  gsLast  pts  sog
propId count   60  142   5    140       73       24     140   60   54


In [4]:
#teams = ['GSW','DEN','POR','SAC','TOR','DAL','PHO','CHI','LAL','HOU','MIA','MEM','DET','MIL','NOP','MIN','CLE','OKC','LAC','BRK','SAS','NYK','WAS','CHO','UTA','IND','BOS','PHI','ATL','ORL']
scraper.get_bref_pos_estimates(
        base_url = 'https://www.basketball-reference.com/teams/{team}/{season}.html#pbp', 
        today_date = today,
        season = season_int,
        database_table = 'brefmisc',
        team_overrides = None
)
missing_teams = scraper.scrape_errors['brefmisc']['url']
if len(missing_teams):
        print('missing:', missing_teams)
        for i in missing_teams:
                scraper.get_bref_pos_estimates(
                        base_url = 'https://www.basketball-reference.com/teams/{team}/{season}.html#pbp', 
                        today_date = today,
                        season = season_int,
                        database_table = 'brefmisc',
                        team_overrides = [i[0]]
                )
    

bref player pos estimates scraped...
missing: [['DEN', 2025], ['MIN', 2025]]
bref player pos estimates scraped...
bref player pos estimates scraped...


In [ ]:
print(today, 'run date...\n')
scraper.get_nba_team_playtype_data(
    base_url = 'https://www.nba.com/stats/teams/{playtype}?TypeGrouping={sideofball}&SeasonType={type}',
    play_types = [
        'isolation', 'transition', 'ball-handler', 'roll-man', 'playtype-post-up',
        'spot-up', 'hand-off', 'cut', 'off-screen','putbacks'
    ],
    sides = ['offensive', 'defensive'],
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    database_table = 'statsteamplaytypes'
)

scraper.get_nba_team_shotzone_data(
    base_url = 'https://www.nba.com/stats/teams/{sideOfBall}?DistanceRange=By+Zone&LastNGames={lastNgames}&SeasonType={type}&DateTo={endDate}', 
    sides = {'offensive':'shooting', 'defensive':'opponent-shooting'},
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    lastNgames = 0,#lastNgames,
    database_table = 'statsteamshotzones',
    dateRange = dateRange
)

scraper.get_nba_team_stats(
        base_url = 'https://www.nba.com/stats/teams/{stats}?SeasonType={seasonType}',
        stats = ['traditional', 'advanced', 'opponent'],
        season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        lastNgames = 0,
        database_table = 'statsteam',
        endDate = None
)

scraper.get_nba_player_playtype_data(
    base_url = 'https://www.nba.com/stats/players/{playtype}?TypeGrouping={sideofball}&SeasonType={type}', 
    play_types = [
        'isolation', 'transition', 'ball-handler', 'roll-man', 'playtype-post-up',
        'spot-up', 'hand-off', 'cut', 'off-screen','putbacks'
    ],
    sides = ['offensive'],
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    database_table = 'statsplayerplaytypes'
)

scraper.get_nba_player_shotzone_data(
    #base_url = 'https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&LastNGames={lastNgames}&SeasonType={type}&DateTo={endDate}', 
    base_url = 'https://www.nba.com/stats/players/shooting?DistanceRange=By+Zone&SeasonType={type}&DateFrom={startDate}&DateTo={endDate}&PerMode=Totals',  
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    lastNgames = 0,
    database_table = 'statsplayershotzones',
    dateRange = dateRange
)

scraper.get_nba_player_passing_data(
    base_url = 'https://www.nba.com/stats/players/passing?DateFrom={d1}&DateTo={d2}&PerMode=Totals&SeasonType={type}', 
    run_date = run_date,
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    database_table = 'statsplayerpassing'
)

scraper.get_nba_player_rebounding_data(
    base_url = 'https://www.nba.com/stats/players/rebounding?DateFrom={d1}&DateTo={d2}&PerMode=Totals&SeasonType={type}', 
    run_date = run_date,
    season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
    database_table = 'statsplayerrebounding'
)

if scraper.scrape_error_flag:
    print(scraper.scrape_errors)

nba team shot zone scraped...


In [ ]:
#prop_scraper.tryMissingProps()
# this didn't work as expected, 
#### added SOG to the nba scrape_errors
#### added new key-value pair for the missing prop:  'missing_dates': ['sog', '2025-01-02', 0]

In [ ]:
prop_scraper.scrape_errors

# loading misscraped passing and rebound date 

In [ ]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import leaguegamefinder

In [22]:
cstring = scraper.pymysql_conn_str
# qery db to get dates of messed up dates
df = pd.read_sql_query(
    #sql = """SELECT date, COUNT(PLAYER_NAME) n FROM statsplayershotzones GROUP BY date HAVING date > '2024-10-20' and n = 50 ORDER BY date;""",
    sql = "SELECT DISTINCT date FROM statsplayershotzones WHERE date > '2024-10-20';",
    con = cstring
)
#dates = df[df['n'] == 50]['date'].tolist()
dates = df['date'].tolist()


In [ ]:
 # Fetch all games for the current season
seasons = '2023-24'
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=seasons, league_id_nullable='00')
games = gamefinder.get_data_frames()[0]

##### for 24-25 season: 12024 = preseason, 22024=reg, 32024=Allstar weekend , 
##### 42024 = playoffs, 52024 = play-ins, 62024 = IST championship 
games.loc[:,'GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
games = games[games['SEASON_ID'].astype(str).str.startswith(('2', '4', '5'))]

In [98]:
for i in dd:
    print(i)
    d = pd.to_datetime(i)
    dateRange = [d, d]
    scraper.get_nba_team_shotzone_data(
        base_url = 'https://www.nba.com/stats/teams/{sideOfBall}?DistanceRange=By+Zone&SeasonType={type}&DateFrom={startDate}&DateTo={endDate}&PerMode=Totals', 
        sides = {'offensive':'shooting', 'defensive':'opponent-shooting'},
        season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        database_table = 'statsteamshotzones',
        dateRange = dateRange
    )


2024-10-22
nba team shot zone scraped 4 offensive,  4 defensive loaded...
2024-10-23
nba team shot zone scraped 20 offensive,  20 defensive loaded...
2024-10-24
nba team shot zone scraped 8 offensive,  8 defensive loaded...
2024-10-25
nba team shot zone scraped 20 offensive,  20 defensive loaded...
2024-10-26
nba team shot zone scraped 20 offensive,  20 defensive loaded...
2024-10-27
nba team shot zone scraped 10 offensive,  10 defensive loaded...
2024-10-29
nba team shot zone scraped 8 offensive,  8 defensive loaded...
2024-10-30
nba team shot zone scraped 22 offensive,  22 defensive loaded...
2024-10-31
nba team shot zone scraped 8 offensive,  8 defensive loaded...
2024-11-01
nba team shot zone scraped 18 offensive,  18 defensive loaded...
2024-11-02
nba team shot zone scraped 20 offensive,  20 defensive loaded...
2024-11-03
nba team shot zone scraped 6 offensive,  6 defensive loaded...
2024-11-04
nba team shot zone scraped 30 offensive,  30 defensive loaded...
2024-11-06
nba team sh

In [ ]:

for i in dd:
    print(i)
    d = pd.to_datetime(i)
    scraper.get_nba_player_passing_data(
        base_url = 'https://www.nba.com/stats/players/passing?DateFrom={d1}&DateTo={d2}&PerMode=Totals&SeasonType={type}', 
        today_date = d,
        day_adjuster = 0,
        season_type = 'Regular+Season',  # ['Regular+Season', 'PlayIn', 'Playoffs']
        #lastNgames = 10,
        database_table = 'statsplayerpassing'
    )

2024-11-25
nba player passing scraped 195 loaded...
2024-11-27
nba player passing scraped 306 loaded...
2024-11-29
nba player passing scraped 206 loaded...
2024-11-30
nba player passing scraped 108 loaded...
2024-12-01
nba player passing scraped 207 loaded...
2024-12-02
nba player passing scraped 88 loaded...
2024-12-03
nba player passing scraped 242 loaded...
2024-12-04
nba player passing scraped 129 loaded...
2024-12-05
nba player passing scraped 173 loaded...
2024-12-06
nba player passing scraped 147 loaded...
2024-12-07
nba player passing scraped 133 loaded...
2024-12-08
nba player passing scraped 254 loaded...
2024-12-09
drop down not found....
nba player passing scraped 17 loaded...
2024-12-10
drop down not found....
nba player passing scraped 43 loaded...
2024-12-11
drop down not found....
nba player passing scraped 37 loaded...
2024-12-12
nba player passing scraped 66 loaded...
2024-12-13
nba player passing scraped 170 loaded...
2024-12-14
drop down not found....
nba player pas

# scratch

In [29]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time 

In [30]:
service = Service('..\\..\\browser\\geckodriver.exe')
driver = webdriver.Firefox(service=service)
url = 'https://www.nba.com/stats/players/rebounding?DateFrom=10%2F24%2F2024&DateTo=10%2F24%2F2024'
driver.get(url)

In [31]:
driver.close()

In [76]:
service = Service('..\\..\\browser\\geckodriver.exe')
driver = webdriver.Firefox(service=service)
# loop through each team webpage to gather data        
base_url = 'https://www.basketball-reference.com/teams/{team}/{season}.html#pbp'

season = 2025

#all_team_data = []
url_errors = []

# basketball ref team url abbrevs
bref_team_abbr = [
    'GSW','DEN','POR', 'SAC','TOR',
    'DAL','PHO','CHI','LAL','HOU',
    'MIA','MEM','DET','MIL','NOP','MIN',
    'CLE','OKC','LAC','BRK','SAS','NYK','WAS','CHO',
    'UTA','IND','BOS','PHI','ATL','ORL'
]

# col names in the database
bref_cols = [
    'player', 'age', 'pos', 'gp', 'gs', 'mp', 'PG', 'SG', 'SF',
    'PF', 'C', 'onCourtPlusMinusPer100', 'onOffPlusMinusPer100',
    'badPass', 'lostBall', 'shootFoulCommitted', 'offFoulCommitted',
    'shootFoulDrawn', 'offFoulDrawn', 'ptsGenFromAst', 'andOnes', 'shotsBlk', 'awards',
    'date', 'team'
]

all_team_data = pd.DataFrame(columns=bref_cols)

# loop through each team webpage to gather data        
for i in bref_team_abbr:
    time.sleep(2)
    try:
        url = base_url.format(team = i, season = str(season))
        driver.get(url)
        time.sleep(2)
        
        #try:
        #    popup = driver.find_element(By.ID, 'modal-close')
            # Wait for the popup to be present and visible
            #popup = WebDriverWait(driver, 3).until(
            #    EC.visibility_of_element_located(By.ID, 'modal-close')
            #)
        #    popup.click()
        #except: continue

        table_id = 'pbp_stats'
        table = None  # Placeholder for the table element
        scroll_attempts = 30  # Number of scrolling attempts
        scroll_step = 500  # Pixels to scroll down on each attempt
  
        # this was removed out of the loop below. loop stopped working and selenium is now able to find it without scrolling
        table = driver.find_element(By.ID, table_id)
        driver.execute_script("arguments[0].scrollIntoView();", table)

        table_html = table.get_attribute('outerHTML')  # Get table HTML
        df = pd.read_html(table_html)[0]  # Convert to DataFrame
        df = df.iloc[:,1:].reset_index(drop=True)
        df.loc[:,'date'] = '2025'
        df.loc[:,'team'] = i
        df.columns = bref_cols

        all_team_data = pd.concat([all_team_data, df])
    
    except:
        print('error', i)
        continue
    

driver.close()
#bref_pos_estimates = pd.DataFrame(all_team_data, columns = bref_cols)

C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\2715966764.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]  # Convert to DataFrame
C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\2715966764.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_team_data = pd.concat([all_team_data, df])


error DEN


C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\2715966764.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]  # Convert to DataFrame
C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\2715966764.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]  # Convert to DataFrame


error TORDAL


C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\2715966764.py:58: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]  # Convert to DataFrame


error CHI
error LAL
error HOU
error MIA
error MEM
error DET
error MIL
error NOP
error MIN
error CLE
error OKC
error LAC


KeyboardInterrupt: 

In [75]:
for i in all_team_data.team.unique():
    print(i)

GSW
DEN
POR
SAC
PHO
CHI
LAL
HOU
MIA
MEM
DET
NOP
MIN
CLE
OKC
LAC
BRK
SAS
NYK
CHO
UTA
IND
BOS
PHI
ATL
ORL


In [41]:
driver.close()

In [29]:
popup = driver.find_element(By.ID, 'modal-close')
popup.click()

In [31]:
# this was removed out of the loop below. loop stopped working and selenium is now able to find it without scrolling
table = driver.find_element(By.ID, table_id)

driver.execute_script("arguments[0].scrollIntoView();", table)

In [32]:
table_html = table.get_attribute('outerHTML')  # Get table HTML
df = pd.read_html(table_html)[0]  # Convert to DataFrame
df = df.iloc[:,1:].reset_index(drop=True)

C:\Users\jrbrz\AppData\Local\Temp\ipykernel_17592\4285168731.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table_html)[0]  # Convert to DataFrame


In [34]:
table

<selenium.webdriver.remote.webelement.WebElement (session="640c6a3f-f302-445d-a3ed-fa522d4d8c12", element="bea42cb3-e84f-4cba-b374-3502da78fdf0")>